In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import onnx
import os
from onnx import optimizer

# Preprocessing: load the model contains two transposes.
model_path = os.path.join('resources', 'two_transposes.onnx')
original_model = onnx.load(model_path)

print('The model before optimization:\n\n{}'.format(onnx.helper.printable_graph(original_model.graph)))

The model before optimization:

graph two-transposes (
  %X[FLOAT, 2x3x4]
) {
  %Y = Transpose[perm = [1, 0, 2]](%X)
  %Z = Transpose[perm = [1, 0, 2]](%Y)
  return %Z
}


In [2]:
# A full list of supported optimization passes can be found using get_available_passes()
all_passes = optimizer.get_available_passes()
print("Available optimization passes:")
for p in all_passes:
    print('\t{}'.format(p))
print()

# Pick one pass as example
passes = ['fuse_consecutive_transposes']

# Apply the optimization on the original serialized model
optimized_model = optimizer.optimize(original_model, passes)

print('The model after optimization:\n\n{}'.format(onnx.helper.printable_graph(optimized_model.graph)))

Available optimization passes:
	eliminate_deadend
	eliminate_identity
	eliminate_nop_dropout
	eliminate_nop_monotone_argmax
	eliminate_nop_pad
	eliminate_nop_transpose
	eliminate_unused_initializer
	extract_constant_to_initializer
	fuse_add_bias_into_conv
	fuse_bn_into_conv
	fuse_consecutive_concats
	fuse_consecutive_log_softmax
	fuse_consecutive_reduce_unsqueeze
	fuse_consecutive_squeezes
	fuse_consecutive_transposes
	fuse_matmul_add_bias_into_gemm
	fuse_pad_into_conv
	fuse_transpose_into_gemm
	lift_lexical_references
	nop
	split_init
	split_predict

The model after optimization:

graph two-transposes (
  %X[FLOAT, 2x3x4]
) {
  %Z = Transpose[perm = [0, 1, 2]](%X)
  return %Z
}
